In [1]:
#Download Data from yt and other sites #??? SHOULD INCLUDE THIS OR NOT?
# Merge all data in one file #??? SHOULD INCLUDE THIS OR NOT?
#Create dataset for learning (Left-Hand,Right-Hand and 12 point from pose) #??? SHOULD INCLUDE THIS OR NOT?

#WLASL VIDEOS WERE GIVEN 
#MLASL Videos Were downloaded and processed
#MLASL VIDEOS WERE CONVERTED TO HAVE 25 fps rate


# !pip install mediapipe
import json
import pandas as pd
import mediapipe as mp
import numpy as np
import cv2
import time
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose

In [3]:
def mediapipe_detection(image, holistic,hands):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR to RGB
    image.flags.writeable = False                  # Image is no longer writeable
    holistic_model = holistic.process(image)                 # Make prediction
    hands_model = hands.process(image)
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    
    resize_up = cv2.resize(image, (1024,800), interpolation= cv2.INTER_AREA)

    return resize_up, holistic_model,hands_model

In [4]:
def draw_landmarks(image, pose,hands):
    if(pose.pose_world_landmarks):
        mp_drawing.draw_landmarks(image, pose.pose_world_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
            # Draw pose connections
        mp_drawing.draw_landmarks(image, pose.pose_world_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    if hands.multi_hand_landmarks:
        for hand_landmarks in hands.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw hands
                # Draw right hand connections  
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
    
def draw_styled_landmarks(image, pose,hands):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_world_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def convert_pose_to_array(hollistic):
    landmarks = hollistic.pose_landmarks;
    data = []
    
    if(landmarks == None):
        data=None
    else:
        landmarks = landmarks.landmark[11:23];
        for index,item in enumerate(landmarks):
            if(item==None):
                data.append(None)
            else:
                landmark = {'X':item.x,'Y':item.y}
                # print(landmark)
                data.append(landmark)
    return data


def convert_pose_world_to_array(hollistic):
    landmarks = hollistic.pose_world_landmarks
    data = []
    
    if(landmarks == None):
        data=None
    else:
        landmarks = landmarks.landmark[11:23];
        for index,item in enumerate(landmarks):
            if(item==None):
                data.append(None)
            else:
                landmark = {'X':item.x,'Y':item.y}
                # print(landmark)
                data.append(landmark)
    return data

def convert_hands_data_to_array(hollistic):
    left_hand_landmarks = hollistic.left_hand_landmarks;
    right_hand_landmarks = hollistic.right_hand_landmarks;

    left_hand = []
    right_hand = []
    
    if(left_hand_landmarks == None):
        left_hand=None
    else:
        left_hand_landmarks = left_hand_landmarks.landmark
        for index,item in enumerate(left_hand_landmarks):
            if(item==None):
                data.append(None)
            else:
                landmark = {'X':item.x,'Y':item.y}
                # print(landmark)
                left_hand.append(landmark)
            
    if(right_hand_landmarks == None):
        right_hand=None
    else:      
        right_hand_landmarks = right_hand_landmarks.landmark
        for index,item in enumerate(right_hand_landmarks):
            if(item==None):
                data.append(None)
            else:
                landmark = {'X':item.x,'Y':item.y}
                # print(landmark)
                right_hand.append(landmark)
            
            
    return [left_hand,right_hand]


def convert_hands_world_to_array(hands):
    hands_position = hands.multi_handedness
    hands_landmarks = hands.multi_hand_world_landmarks
    left_hand = []
    right_hand = []
    
    if(hands_position == None):
        left_hand.append(None)
        right_hand.append(None)
    elif (len(hands_position) == 1):
        detected_hand = hands_position[0].classification[0].label
        landmarks = hands_landmarks[0].landmark
        for index,item in enumerate(landmarks):
            landmark = {'X':item.x,'Y':item.y,'Z':item.z}
            if(detected_hand=="Left"):
                right_hand.append(landmark)
                left_hand.append(None)
            else:
                left_hand.append(landmark)
                right_hand.append(None)
    else:
        first_hand = hands_position[0].classification[0]
        second_hand = hands_position[1].classification[0]
        if(first_hand.label == second_hand.label):
            landmarks = []
            if(first_hand.score>second_hand.score):
                landmarks = hands_landmarks[0].landmark
            else:
                landmarks = hands_landmarks[1].landmark
            for index,item in enumerate(landmarks):
                landmark = {'X':item.x,'Y':item.y,'Z':item.z}
                if(first_hand.label=="Left"):
                    right_hand.append(landmark)
                    left_hand.append(None)
                else:
                    left_hand.append(landmark)
                    right_hand.append(None)
        else:
            left_hand_landmarks = hands_landmarks[0].landmark
            for index,item in enumerate(left_hand_landmarks):
                landmark = {'X':item.x,'Y':item.y,'Z':item.z}
                right_hand.append(landmark)
                
            right_hand_landmarks = hands_landmarks[1].landmark
            for index,item in enumerate(right_hand_landmarks):
                landmark = {'X':item.x,'Y':item.y,'Z':item.z}
                left_hand.append(landmark)
            
    return [left_hand,right_hand]
    

In [5]:
data = pd.read_json("Data.json")
X_book = data.iloc[0:14]
X_drink = data.iloc[40:54]
X_computer = data.iloc[75:89]
data_type = {"right_hand": [], "left_hand":[],"pose":[]}
X = np.concatenate((X_book,X_drink,X_computer))


In [6]:

index = 0;
for item in X:
    print(item)
    print(Folder + str(item[1]).zfill(5) + ".mp4")
    cap = cv2.VideoCapture(Folder + str(item[1]).zfill(5) + ".mp4")
    right_hand = []
    left_hand = []
    pose_shape = []
    pose_world = []
    right_hand_world = []
    left_hand_world = []
    data_type["Label"].append(item[0].lower())
    print(index)
    index = index+1
    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        with mp_hands.Hands(min_detection_confidence=0.3,min_tracking_confidence=0.3) as hands:
            while(cap.isOpened()):
                success, frame = cap.read()
                if(not success):
                    break
                if(success):
                    frame.flags.writeable = False
                    image,hollistic,hands_position = mediapipe_detection(frame,pose,hands)
                    pose_shape.append(convert_pose_to_array(hollistic))
                    pose_world.append(convert_pose_world_to_array(hollistic))
                    
                    
                    hands_landmarks = convert_hands_data_to_array(hollistic)
                    hands_world_landmarks = convert_hands_world_to_array(hands_position)

                    left_hand.append(hands_landmarks[0])
                    right_hand.append(hands_landmarks[1])

                    left_hand_world.append(hands_world_landmarks[0])
                    right_hand_world.append(hands_world_landmarks[1])

#                     data_type["POSE_LANDMARKS"].append(convert_pose_to_array(hollistic))
#                     data_type["POSE_WORLD_LANDMARKS"].append(convert_pose_to_array(hollistic))

#                     hands_landmarks = convert_hands_data_to_array(hollistic)
#                     data_type["RIGHT_HAND_LANDMARKS"].append(hands_landmarks[1])
#                     data_type["LEFT_HAND_LANDMARKS"].append(hands_landmarks[0])
#                     hands_world_landmarks = convert_hands_world_to_array(hands_position)
                    
#                     data_type["RIGHT_HAND_WORLD_LANDMARKS"].append(hands_world_landmarks[1])
#                     data_type["LEFT_HAND_WORLD_LANDMARKS"].append(hands_world_landmarks[0])
    data_type["POSE_LANDMARKS"].append(pose_shape)
    data_type["POSE_WORLD_LANDMARKS"].append(pose_world)
    
    data_type["RIGHT_HAND_LANDMARKS"].append(right_hand)
    data_type["LEFT_HAND_LANDMARKS"].append(left_hand)
    
    data_type["LEFT_HAND_WORLD_LANDMARKS"].append(left_hand_world)
    data_type["RIGHT_HAND_WORLD_LANDMARKS"].append(right_hand_world)

['book' 69241 25 'http://aslbricks.org/New/ASL-Videos/book.mp4']
WLASL2000/WLASL2000/69241.mp4
0
['book' 65225 25
 'https://aslsignbank.haskins.yale.edu/dictionary/protected_media/glossvideo/ASL/BO/BOOK-418.mp4']
WLASL2000/WLASL2000/65225.mp4
1
['book' 68011 25 'https://www.youtube.com/watch?v=0UsjUE-TXns']
WLASL2000/WLASL2000/68011.mp4
2
['book' 68208 25 'https://www.youtube.com/watch?v=1QOYOZ3g-aY']
WLASL2000/WLASL2000/68208.mp4
3
['book' 68012 25 'https://www.youtube.com/watch?v=aGtIHKEdCds']
WLASL2000/WLASL2000/68012.mp4
4
['book' 70212 25 'https://www.youtube.com/watch?v=hjS0dQDgbjo']
WLASL2000/WLASL2000/70212.mp4
5
['book' 70266 25 'https://www.youtube.com/watch?v=WGfiiDgrq1I']
WLASL2000/WLASL2000/70266.mp4
6
['book' 7085 25 'http://www.aslpro.com/main/b/book_english_grammar.swf']
WLASL2000/WLASL2000/07085.mp4
7
['book' 7086 25 'http://www.aslpro.com/main/b/book_geography.swf']
WLASL2000/WLASL2000/07086.mp4
8
['book' 7087 25 'http://www.aslpro.com/main/b/book_geometry.swf']
WLASL

In [31]:
df = pd.DataFrame.from_dict(data_type)
df.to_json('testData.json',orient='index')

# read_df = pd.read_json('testData.json')


In [35]:
# read_df = pd.read_json(open("testData.json", "r", encoding="utf8"))
read_df = pd.read_json('testData.json',orient='index')



In [53]:
len(read_df)

42

In [61]:
def convert_to_50_frames(data):
    for item in data.iterrows():
        print(item[1].RIGHT_HAND_LANDMARKS)
        
convert_to_50_frames(read_df)

[None, None, None, None, None, [{'X': 0.35551533100000005, 'Y': 0.864187479}, {'X': 0.3960908651, 'Y': 0.8497877717000001}, {'X': 0.4400761127, 'Y': 0.8626277447}, {'X': 0.4674811065, 'Y': 0.8834815621000001}, {'X': 0.4918919802, 'Y': 0.8970381618000001}, {'X': 0.44576868410000003, 'Y': 0.8973474503000001}, {'X': 0.477273345, 'Y': 0.9152846336}, {'X': 0.4989069104, 'Y': 0.9311358929}, {'X': 0.5153593421, 'Y': 0.9445013404}, {'X': 0.4290343225, 'Y': 0.9150829911}, {'X': 0.4590526521, 'Y': 0.9349609613000001}, {'X': 0.4793488681, 'Y': 0.9489623308}, {'X': 0.496537596, 'Y': 0.9639623761}, {'X': 0.41192710400000004, 'Y': 0.927231133}, {'X': 0.43724152450000003, 'Y': 0.9469957352}, {'X': 0.45616725090000004, 'Y': 0.9611237645}, {'X': 0.47318315510000003, 'Y': 0.9759463072000001}, {'X': 0.394654125, 'Y': 0.9351946115}, {'X': 0.41558665040000003, 'Y': 0.9553822279}, {'X': 0.4310359359, 'Y': 0.9694470763}, {'X': 0.4451486766, 'Y': 0.9821882844000001}], [{'X': 0.3504456282, 'Y': 0.8286206722}, 